In [5]:
"""
ETL – Salários em Portugal
"""

from pathlib import Path
import pandas as pd

# =========================================================
# 1. CONFIGURAÇÃO DE PASTAS
# =========================================================
BASE_RAW = Path("../data/raw")        # onde estão os CSV originais
BASE_OUT = Path("../data/processed")  # onde serão salvos os CSV limpos
BASE_OUT.mkdir(parents=True, exist_ok=True)


# =========================================================
# 2. FUNÇÕES AUXILIARES
# =========================================================
def load_pordata(path: Path) -> pd.DataFrame:
    """
    Lê um ficheiro Pordata e padroniza os nomes de colunas.
    """
    df = pd.read_csv(path, encoding="utf-8")

    rename_map = {}
    for col in df.columns:
        new = col

        if col.startswith("01. Ano"):
            new = "ano"
        elif "Nome Regi" in col:
            new = "regiao"
        elif "Nome Pa" in col:
            new = "pais"
        elif "Âmbito Geogr" in col or "Ãmbito Geogr" in col:
            new = "ambito"
        elif "Indicador" in col:
            new = "indicador"
        elif "Filtro 1" in col:
            new = "filtro1"
        elif "Filtro 2" in col:
            new = "filtro2"
        elif "Filtro 3" in col:
            new = "filtro3"
        elif "Escala" in col:
            new = "escala"
        elif "Símbolo" in col or "SÃ­mbolo" in col:
            new = "simbolo"
        elif "Valor" in col:
            new = "valor"

        rename_map[col] = new

    df = df.rename(columns=rename_map)

    # ano como inteiro
    if "ano" in df.columns:
        df["ano"] = pd.to_numeric(df["ano"], errors="coerce").astype("Int64")

    # valor numérico
    if "valor" in df.columns:
        df["valor"] = (
            df["valor"].astype(str)
            .str.replace(",", ".", regex=False)
        )
        df["valor"] = pd.to_numeric(df["valor"], errors="coerce")
        df = df.dropna(subset=["valor"])

    return df


def filter_portugal(df: pd.DataFrame) -> pd.DataFrame:
    """
    Mantém apenas regiao == 'Portugal', se existir essa coluna.
    """
    if "regiao" not in df.columns:
        return df.copy()
    return df[df["regiao"] == "Portugal"].copy()


def filter_euros(df: pd.DataFrame) -> pd.DataFrame:
    """
    Mantém apenas linhas cuja escala contenha 'euro', se a coluna existir.
    Nas bases 581–588 isso garante valores em euros.
    """
    if "escala" not in df.columns:
        return df.copy()
    mask = df["escala"].astype(str).str.contains("euro", case=False, na=False)
    return df[mask].copy()


def remove_total(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    """
    Remove linhas com valor 'Total' nas colunas de dimensão informadas.
    """
    out = df.copy()
    for c in cols:
        if c in out.columns:
            out = out[~out[c].astype(str).str.fullmatch("Total", case=False)]
    return out


# =========================================================
# 3. FACT – GANHO MÉDIO MENSAL (581–588)
# =========================================================

# 581 – Ganho médio mensal – total (Portugal)
df = load_pordata(BASE_RAW / "581.csv")
df = filter_portugal(df)
df = filter_euros(df)

fact_ganho_total = df[["ano", "valor"]].copy()
fact_ganho_total = fact_ganho_total.rename(columns={"valor": "ganho_medio_mensal"})
fact_ganho_total.to_csv(BASE_OUT / "fact_ganho_total.csv", index=False)


# 582 – Ganho médio mensal – por escolaridade
df = load_pordata(BASE_RAW / "582.csv")
df = filter_portugal(df)
df = filter_euros(df)
df = df.rename(columns={"filtro1": "escolaridade"})
df = remove_total(df, ["escolaridade"])

fact_ganho_escolaridade = df[["ano", "escolaridade", "valor"]].copy()
fact_ganho_escolaridade = fact_ganho_escolaridade.rename(
    columns={"valor": "ganho_medio_mensal"}
)
fact_ganho_escolaridade.to_csv(
    BASE_OUT / "fact_ganho_escolaridade.csv", index=False
)


# 583 – Ganho médio mensal – por grupo profissional
df = load_pordata(BASE_RAW / "583.csv")
df = filter_portugal(df)
df = filter_euros(df)
df = df.rename(columns={"filtro1": "grupo_profissional"})
df = remove_total(df, ["grupo_profissional"])

fact_ganho_grupo_prof = df[["ano", "grupo_profissional", "valor"]].copy()
fact_ganho_grupo_prof = fact_ganho_grupo_prof.rename(
    columns={"valor": "ganho_medio_mensal"}
)
fact_ganho_grupo_prof.to_csv(
    BASE_OUT / "fact_ganho_grupo_prof.csv", index=False
)


# 584 – Ganho médio mensal – sexo x grande setor de atividade
df = load_pordata(BASE_RAW / "584.csv")
df = filter_portugal(df)
df = filter_euros(df)
df = df.rename(columns={"filtro1": "sexo", "filtro2": "setor_macro"})
df = remove_total(df, ["sexo", "setor_macro"])

fact_ganho_setor_macro = df[["ano", "sexo", "setor_macro", "valor"]].copy()
fact_ganho_setor_macro = fact_ganho_setor_macro.rename(
    columns={"valor": "ganho_medio_mensal"}
)
fact_ganho_setor_macro.to_csv(
    BASE_OUT / "fact_ganho_setor_macro.csv", index=False
)


# 585 – Ganho médio mensal – categoria x setor detalhado
df = load_pordata(BASE_RAW / "585.csv")
df = filter_portugal(df)
df = filter_euros(df)
df = df.rename(columns={"filtro1": "categoria", "filtro2": "setor_detalhado"})
df = remove_total(df, ["categoria", "setor_detalhado"])

fact_ganho_cat_setor = df[["ano", "categoria", "setor_detalhado", "valor"]].copy()
fact_ganho_cat_setor = fact_ganho_cat_setor.rename(
    columns={"valor": "ganho_medio_mensal"}
)
fact_ganho_cat_setor.to_csv(
    BASE_OUT / "fact_ganho_cat_setor.csv", index=False
)


# 586 – Ganho médio mensal – sexo x categoria
df = load_pordata(BASE_RAW / "586.csv")
df = filter_portugal(df)
df = filter_euros(df)
df = df.rename(columns={"filtro1": "sexo", "filtro2": "categoria"})
df = remove_total(df, ["sexo", "categoria"])

fact_ganho_cat_sexo = df[["ano", "sexo", "categoria", "valor"]].copy()
fact_ganho_cat_sexo = fact_ganho_cat_sexo.rename(
    columns={"valor": "ganho_medio_mensal"}
)
fact_ganho_cat_sexo.to_csv(
    BASE_OUT / "fact_ganho_cat_sexo.csv", index=False
)


# 587 – Ganho médio mensal – sexo x idade
df = load_pordata(BASE_RAW / "587.csv")
df = filter_portugal(df)
df = filter_euros(df)
df = df.rename(columns={"filtro1": "sexo", "filtro2": "faixa_etaria"})
df = remove_total(df, ["sexo", "faixa_etaria"])

fact_ganho_idade = df[["ano", "sexo", "faixa_etaria", "valor"]].copy()
fact_ganho_idade = fact_ganho_idade.rename(
    columns={"valor": "ganho_medio_mensal"}
)
fact_ganho_idade.to_csv(
    BASE_OUT / "fact_ganho_idade.csv", index=False
)


# 588 – Ganho médio mensal – por sexo
df = load_pordata(BASE_RAW / "588.csv")
df = filter_portugal(df)
df = filter_euros(df)
df = df.rename(columns={"filtro1": "sexo"})
df = remove_total(df, ["sexo"])

fact_ganho_sexo = df[["ano", "sexo", "valor"]].copy()
fact_ganho_sexo = fact_ganho_sexo.rename(
    columns={"valor": "ganho_medio_mensal"}
)
fact_ganho_sexo.to_csv(
    BASE_OUT / "fact_ganho_sexo.csv", index=False
)


# =========================================================
# 4. FACT – SALÁRIO MÍNIMO PORTUGAL (183)
# =========================================================
df = load_pordata(BASE_RAW / "183.csv")
df = filter_portugal(df)

# Filtrar apenas o indicador:
# "Retribuição mínima mensal garantida (RMMG) - Portugal continental"
# excluindo agricultura e serviço doméstico
mask_base = df["indicador"].astype(str).str.contains(
    "Retribui", case=False, na=False
)
mask_cont = df["indicador"].astype(str).str.contains(
    "Portugal continental", case=False, na=False
)
mask_agric = df["indicador"].astype(str).str.contains(
    "agric", case=False, na=False
)
mask_dom = df["indicador"].astype(str).str.contains(
    "dom", case=False, na=False
)

df = df[mask_base & mask_cont & ~mask_agric & ~mask_dom].copy()

# Excluir anos em que os valores não estão em euros
# Aqui consideramos anos >= 1999 como período em euros
df = df[df["ano"] >= 1999].copy()

fact_salario_minimo_pt = df[["ano", "valor"]].copy()
fact_salario_minimo_pt = fact_salario_minimo_pt.rename(
    columns={"valor": "salario_minimo_mensal"}
)
fact_salario_minimo_pt.to_csv(
    BASE_OUT / "fact_salario_minimo_pt.csv", index=False
)

print("ETL corrigido concluído. Ficheiros limpos salvos em:", BASE_OUT.resolve())


ETL corrigido concluído. Ficheiros limpos salvos em: C:\OneDrive\My Documents\Projetos - Análise de Dados\etl_dashboard_salarios_portugal\data\processed
